<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/customization/llms/AzureOpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# Azure OpenAI


Azure OpenAI资源与标准OpenAI资源不同，因为您无法生成嵌入，除非您使用嵌入模型。支持这些模型的区域可以在此处找到：https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models#embeddings-models

此外，支持嵌入模型的区域不幸不支持OpenAI模型的最新版本（<*>-003），因此我们被迫在一个区域用于嵌入，另一个用于文本生成。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-embeddings-azure-openai
%pip install llama-index-llms-azure-openai

In [ ]:
!pip install llama-index

In [ ]:
来自llama_index.llms.azure_openai的AzureOpenAI
来自llama_index.embeddings.azure_openai的AzureOpenAIEmbedding
来自llama_index.core的VectorStoreIndex和SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG用于更详细的输出
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

在这里，我们设置嵌入模型（用于检索）和llm（用于文本生成）。
请注意，您不仅需要模型名称（例如“text-embedding-ada-002”），还需要模型部署名称（在Azure部署模型时选择的名称）。
在初始化`AzureOpenAI`和`OpenAIEmbedding`时，您必须将部署名称作为参数传递。


In [ ]:
api_key = "<api-key>"
azure_endpoint = "https://<your-resource-name>.openai.azure.com/"
api_version = "2023-07-01-preview"

llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="my-custom-llm",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# 您需要部署自己的嵌入模型以及自己的聊天完成模型
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="my-custom-embedding",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
documents = SimpleDirectoryReader(
    input_files=["../../data/paul_graham/paul_graham_essay.txt"]
).load_data()
index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


In [ ]:
query = "What is most interesting about this essay?"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

print(answer.get_formatted_sources())
print("query was:", query)
print("answer was:", answer)

INFO:httpx:HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-embedding/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://test-simon.openai.azure.com//openai/deployments/my-custom-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
> Source (Doc id: 3e0d1e3f-9099-483f-9abd-8f352c